In [6]:
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import KFold
from core import criar_modelo, ParImageGenerator, create_tensorboard_callback
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping

In [7]:
tf.config.run_functions_eagerly(True)

os.makedirs('../model', exist_ok=True)
os.makedirs('../results', exist_ok=True)
os.makedirs('../logs', exist_ok=True)

In [8]:
# ============================
# 🔁 Carregar todos os dados
# ============================
gen_temporario = ParImageGenerator('../image/treinamento', batch_size=1, augmentacao=False)
dados = list(zip(gen_temporario.imagens, gen_temporario.labels))  # manter como lista

In [9]:
# ============================
# 🔁 K-Fold Cross-Validation
# ============================
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [10]:
fold = 1
accuracies = []
losses = []

log_path = '../results/cv_info.txt'
with open(log_path, 'w') as log_file:

    for train_idx, val_idx in kf.split(dados):
        log_file.write(f"[Fold {fold}]\n")
        log_file.write(f"Treino: {list(train_idx)}\n")
        log_file.write(f"Validação: {list(val_idx)}\n\n")

        print(f"\n🌀 Treinando Fold {fold}...")

        train_data = [dados[i] for i in train_idx]
        val_data = [dados[i] for i in val_idx]

        train_gen = ParImageGenerator(dados=train_data, batch_size=8, augmentacao=True)
        val_gen = ParImageGenerator(dados=val_data, batch_size=8, augmentacao=False)

        modelo = criar_modelo()
        modelo.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                        loss='binary_crossentropy',
                        metrics=['accuracy'])

        checkpoint = ModelCheckpoint(
            filepath=f'../model/melhor_modelo_fold{fold}.h5',
            monitor='val_accuracy',
            save_best_only=True, # salva o melhor modelo
            verbose=1
        )

        logger = CSVLogger(f'../results/historico_treinamento_fold{fold}.csv', append=False)

        early_stopping = EarlyStopping(
            monitor='val_loss', # Monitore a perda de validação
            patience=7,        # Número de épocas sem melhora antes de parar
            restore_best_weights=True, # Restaura os pesos do melhor epoch
            verbose=1
        )

        os.makedirs(f'../logs/fold_{fold}', exist_ok=True)
        tensorboard_callback = create_tensorboard_callback(fold_name=f'fold_{fold}')

        modelo.fit(
            train_gen,
            validation_data=val_gen,
            # epochs=15,
            epochs=1,
            # steps_per_epoch=len(train_gen),
            steps_per_epoch=1,
            callbacks=[checkpoint, logger, early_stopping, tensorboard_callback],
            verbose=1
        )

        loss, accuracy = modelo.evaluate(val_gen, verbose=1)
        print(f"📊 [Fold {fold}] Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

        accuracies.append(accuracy)
        losses.append(loss)
        fold += 1


🌀 Treinando Fold 1...
✅ Logs do TensorBoard serão salvos em: c:\cod\python\AI\logs\fold_1


c:\cod\python\AI\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
c:\cod\python\AI\venv\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11s/step - accuracy: 1.0000 - loss: 0.2316
Epoch 1: val_accuracy improved from -inf to 0.80357, saving model to ../model/melhor_modelo_fold1.h5


1/1 ━━━━━━━━━━━━━━━━━━━━ 47s 47s/step - accuracy: 1.0000 - loss: 0.2316 - val_accuracy: 0.8036 - val_loss: 0.4999
Restoring model weights from the end of the best epoch: 1.
14/14 ━━━━━━━━━━━━━━━━━━━━ 23s 2s/step - accuracy: 0.8572 - loss: 0.4392
📊 [Fold 1] Loss: 0.5000, Accuracy: 0.8036

🌀 Treinando Fold 2...
✅ Logs do TensorBoard serão salvos em: c:\cod\python\AI\logs\fold_2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10s/step - accuracy: 0.2500 - loss: 0.8968
Epoch 1: val_accuracy improved from -inf to 0.34821, saving model to ../model/melhor_modelo_fold2.h5


1/1 ━━━━━━━━━━━━━━━━━━━━ 52s 52s/step - accuracy: 0.2500 - loss: 0.8968 - val_accuracy: 0.3482 - val_loss: 0.7019
Restoring model weights from the end of the best epoch: 1.
14/14 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.3951 - loss: 0.7010
📊 [Fold 2] Loss: 0.7022, Accuracy: 0.3393

🌀 Treinando Fold 3...
✅ Logs do TensorBoard serão salvos em: c:\cod\python\AI\logs\fold_3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 0.5000 - loss: 0.7855
Epoch 1: val_accuracy improved from -inf to 0.13393, saving model to ../model/melhor_modelo_fold3.h5


1/1 ━━━━━━━━━━━━━━━━━━━━ 46s 46s/step - accuracy: 0.5000 - loss: 0.7855 - val_accuracy: 0.1339 - val_loss: 1.1211
Restoring model weights from the end of the best epoch: 1.
14/14 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.1080 - loss: 1.1438
📊 [Fold 3] Loss: 1.1210, Accuracy: 0.1339

🌀 Treinando Fold 4...
✅ Logs do TensorBoard serão salvos em: c:\cod\python\AI\logs\fold_4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9s/step - accuracy: 0.0000e+00 - loss: 1.2610
Epoch 1: val_accuracy improved from -inf to 0.11607, saving model to ../model/melhor_modelo_fold4.h5


1/1 ━━━━━━━━━━━━━━━━━━━━ 40s 40s/step - accuracy: 0.0000e+00 - loss: 1.2610 - val_accuracy: 0.1161 - val_loss: 0.9850
Restoring model weights from the end of the best epoch: 1.
14/14 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.1536 - loss: 0.9624
📊 [Fold 4] Loss: 0.9906, Accuracy: 0.1071

🌀 Treinando Fold 5...
✅ Logs do TensorBoard serão salvos em: c:\cod\python\AI\logs\fold_5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19s/step - accuracy: 0.8750 - loss: 0.4643
Epoch 1: val_accuracy improved from -inf to 0.11607, saving model to ../model/melhor_modelo_fold5.h5


1/1 ━━━━━━━━━━━━━━━━━━━━ 121s 121s/step - accuracy: 0.8750 - loss: 0.4643 - val_accuracy: 0.1161 - val_loss: 1.6686
Restoring model weights from the end of the best epoch: 1.
14/14 ━━━━━━━━━━━━━━━━━━━━ 42s 3s/step - accuracy: 0.1136 - loss: 1.6731
📊 [Fold 5] Loss: 1.6531, Accuracy: 0.1250


In [11]:
# ============================
# 📈 Resultados finais
# ============================
media_acc = np.mean(accuracies)
media_loss = np.mean(losses)

In [12]:
print(f"\n✅ Cross-validation finalizada!")
print(f"📉 Média da Loss: {media_loss:.4f}")
print(f"✅ Média da Accuracy: {media_acc:.4f}")


✅ Cross-validation finalizada!
📉 Média da Loss: 0.9934
✅ Média da Accuracy: 0.3018


In [13]:
# Salvar resumo no log
with open(log_path, 'a') as log_file:
    log_file.write("=== Resultado Final ===\n")
    log_file.write(f"Média da Loss: {media_loss:.4f}\n")
    log_file.write(f"Média da Accuracy: {media_acc:.4f}\n")